# FLIP (02) Optimization Data Science

---
Team Director: Meng Ren | mren@tulip.academy<br />

TULIP Academy <br />
http://www.tulip.academy 

---

Stability
=========

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
rcParams['figure.figsize'] = (12,6)

Evidence
--------

### Partial differential equations

Partial differential equations (PDEs) involve derivatives of functions
of more than one variable, say $u(x, y)$ or $y(t,
  x)$. Hence more complex behaviour and more interesting physics.

Only look at linear problems. Also only consider finite difference
methods: simple to analyse but not always competitive.

Previously looked at simple finite difference algorithms applied to heat
and advection equations. At present have no explanation for which worked
and which did not; for that we need stability theory.

### The results

   Algorithm |    Heat equation   |  Advection equation
  :---------:|:------------------:|:--------------------:
    FTCS     |  Stable $s < 1/2$  |       Unstable
    BTCS     | Stable $\forall s$ |          ??
    FTBS     |         ??         |    Stable $c < 1$

Here $s = k \delta / h^2$ for the heat equation

$$\partial_t y - k \partial_{x x} y = 0,$$

and $c = v \delta / h$ for the advection equation

$$\partial_t y + v \partial_x y = 0.$$

Lax Theorem
-----------

### Consistency

 Theory for multistep methods for IVPs related
*consistency*, *stability* and *convergence*. Have similar concepts and
theory for PDEs.  Finite difference equation
*consistent* if it tends to the PDE as grid sizes go to zero
*independently*. 

Local error must vanish as $h, \delta \rightarrow 0$ in any way.



All the methods defined previously are clearly consistent.


The Lax method for the advection equation,

$$y_i^{n+1} = \frac{1}{2} \left( y_{i+1}^n + y_{i-1}^n \right) -
\frac{c}{2} \left( y_{i+1}^n - y_{i-1}^n \right)$$

can be Taylor expanded to give

$$\partial_t y + v \partial_x y = \tfrac{1}{2} \left(
h^2 / \delta - v^2 \delta \right) \partial_{x x} y + \dots$$

which is consistent if, for example, $h / \delta$ is constant, but not
in general.

### Stability

As before a finite difference equation is *stable* if it produces a
bounded solution and unstable otherwise.

Have seen that BTCS is stable for the heat equation, whilst FTCS was
stable only when $s < 1/2$. For heat equation FTCS is *conditionally*
stable, whilst BTCS is *unconditionally* stable. Have also seen that
FTCS is unstable for advection equation; it is *unconditionally
unstable*.

As for IVPs there may exist genuine unbounded solutions, and spurious
solutions that swamp genuine solutions. Often closely related to the
complex concept of well-posedness: shall ignore this issue here.

### Convergence

Key concept is *convergence*: in the limit of zero grid spacing the
finite difference solution converges to the exact solution.

Exceptionally difficult to directly prove in general. Instead rely on

**Lax’s Theorem (1954)**: Given a properly posed linear initial-value problem and a finite
difference approximation to it that is consistent, stability is
necessary and sufficient for convergence.

Can be extended (with some caveats) to nonlinear problems with
non-trivial boundary conditions, even when discontinuities form.

Von Neumann stability theory
============================

### Determining stability

Von Neumann stability method for linear algorithms and equations:

1.  Make ansatz that solutions of the finite difference equation have form

    $$y_{\ell}^k =  e^{\alpha j \ell h} q^k, \qquad (\, j = \sqrt{-1}, \alpha \in {\mathbb R}, q \in {\mathbb C} ).$$

2.  Substitute this solution into the finite difference equation.

3.  Determine $q$ as a function of $\alpha$, the grid spacings $h,\delta$, and known numbers (such as the advection velocity $v$).

4.  Determine under what conditions we have stability, i.e. when $$|q| < 1 \quad \forall \alpha.$$

Can apply method blindly to produce stability result; but how does
it come about?

### Von Neumann stability

Consider a linear initial-value problem, such as heat
or advection equation. Assume we have a consistent finite difference
algorithm (easily checked). 

To prove convergence need to show algorithm is stable: i.e., numerical
result is bounded.


Assume we can separate variables and write

$$y = \sum_{m = 0}^{\infty} A_{m}(t) \cos (m x) +
B_{m}(t) \sin (m x).$$

Implies assumptions about e.g. spatial domain (ignore
here). 

Use exponential form of trigonometric terms to write as

$$y = \sum_{m = -\infty}^{\infty} a_{m}(t) e^{j m x}$$

where $a$ is *complex* function (containing the two degrees of freedom
in $A,B$), and $j = \sqrt{-1}$.


As problem is linear can superpose solutions, so each $a_{m}$ is
independent. Hence problem stable *only if* $a_{m}$ does not grow with
$t$ for *every* $m$.

### Linear stability

From linearity consider just the single mode “solution”

$$y = a_{m}(t) e^{j m x}, \qquad m \in \mathbb{Z}.$$

Just need to show that $y$ bounded for general $m$.

Consider a point on grid $\{ x_{\ell} \} = \ell h$, so that

$$y_{\ell} = a_{m}(t) e^{j m \ell h}.$$

At first two steps $t^0, t^1 = t^0 + \delta$, have values

$$y_{\ell}^0 = a_{m}(t^0) e^{j m \ell h}, \quad y_{\ell}^1 =
a_{m}(t^1) e^{j m \ell h}.$$

*Define* the growth from one timestep to the next as

$$q \equiv \frac{y_{\ell}^{k+1}}{y_{\ell}^k} =
\frac{a_{m}(t^{k+1})}{a_{m}(t^k)} \quad \therefore \quad
\text{Stability} \Leftrightarrow | q | < 1.$$

Von Neumann stability in practise
=================================

### FTCS for the heat equation

Our finite difference algorithm is

$$y_i^{n+1} = ( 1 - 2 s ) y_i^n + s \left( y_{i+1}^n + y_{i-1}^n
\right).$$

By identifying $i \leftrightarrow \ell$ and $n \leftrightarrow k$ we can
substitute in our ansatz

$$\begin{aligned}
&& y_{\ell}^k & =  e^{\alpha j \ell h} q^k. \\
& \implies & 
e^{\alpha j \ell h} q^{k+1} &= (1 - 2 s) e^{\alpha j \ell h} q^k +
s \left( e^{\alpha j (\ell + 1) h} q^k + e^{\alpha j (\ell - 1) h}
  q^k \right).
\end{aligned}$$

Removing the common factor of $e^{\alpha j \ell h} q^k$ gives

$$q = (1 - 2 s) + s \left( e^{\alpha j h} +  e^{-\alpha j h} \right).$$

Relates $q$ to (undetermined) $\alpha$ and (free) $h$ and $s$.


$$y_i^{n+1} = ( 1 - 2 s ) y_i^n + s \left( y_{i+1}^n + y_{i-1}^n
\right).$$

$$q = (1 - 2 s) + s \left( e^{\alpha j h} +  e^{-\alpha j h} \right).$$

Now bound $|q|$ for *any* $\alpha$ (may depend on grid
spacing). 

More obvious by writing

$$q = 1 - 4 s \sin^2 \left(\frac{\alpha h}{2} \right).$$


See that, for arbitrary $\alpha$, $\sin$ term bounded between $0$ and
$1$.


Therefore the limiting values for $q$ are

$$\begin{aligned}
q & \le 1 & (\sin^2(\dots) & = 0) \\
q & \ge 1 - 4 s & (\sin^2(\dots) & = 1).
\end{aligned}$$


Therefore stability, and hence convergence, is given when

$$|q| < 1 \Leftrightarrow s < \tfrac{1}{2} \Leftrightarrow
\frac{\delta}{h^2} < \tfrac{1}{2}.$$

### BTCS for the heat equation

We look at BTCS in the form

$$(1 + 2 s) y_i^{n+1} = y_i^n + s \left( y_{i+1}^{n+1} +
y_{i-1}^{n+1} \right).$$

Use ansatz $y_{\ell}^k =  \exp (\alpha j \ell h) q^k$ and removing
common factors:

$$(1 + 2 s) q = 1 + s q \left( e^{\alpha j h} +  e^{-\alpha j h}
\right).$$

Rewrite this as

$$\begin{aligned}
&& q & = 1 + 2 s \left( e^{\alpha j h} +  e^{-\alpha j h} - 2 \right)
q \\
&& & = 1 - 4 s q \sin^2 \left( \tfrac{\alpha h}{2} \right) \\
& \implies & q & = \left( 1 + 4 s \sin^2 \left( \tfrac{\alpha
      h}{2} \right) \right)^{-1}, 
\end{aligned}$$

which is *always* less than 1. Hence BTCS is unconditionally stable.

### FTCS for the advection equation

Ignore difference in *differential* equation; only need analysis at
level of finite difference equation:

$$y_i^{n+1} = y_i^n - \frac{c}{2} \left( y_{i+1}^n - y_{i-1}^n
\right).$$

Use ansatz $y_{\ell}^k = \exp (\alpha j \ell h) q^k$ and remove common
factors:

$$\begin{aligned}
&& q & = 1 - \frac{c}{2} \left( e^{\alpha j h} - e^{-\alpha j h}
\right) \\
& \implies & q & = 1 - j c \sin ( \alpha h ).
\end{aligned}$$

As this is complex the modulus is

$$|q|^2 = 1 + c^2 \sin^2 ( \alpha h )$$

which is always greater than 1: method is unconditionally unstable.

### FTBS for the advection equation

Finally for the upwind method, FTBS, we had the difference equation

$$y_i^{n+1} = y_i^n - c \left( y_{i}^n - y_{i-1}^n \right).$$

Use ansatz $y_{\ell}^k =  \exp (\alpha j \ell h) q^k$ and remove common
factors:

$$q = 1 - c \left( 1 - e^{-\alpha j h} \right).$$

Describes circle radius $c$ in the complex plane, centre at $1 -
  c$. If $c > 1$ there are finite values of $\alpha$ for which
$|q| > 1$, leading to instability; if $c < 1$ then $q$ is at most 1,
leading to stability.

Summary
=======

-   Von Neumann stability analysis is the key tool for understanding the results seen “experimentally” in the last lecture.

-   By looking at the growth or decay of individual frequency modes we can check the stability of the method; Lax’s theorem then proves convergence.

-   For complex equations and/or algorithms, it is often very difficult or impossible to apply even Von Neumann stability analysis.

-   If an algorithm does not work in the linear case then it is highly unlikely to work for more complex problems!